# Clone the GitHub Repository.

In [ ]:
!git clone -b dev https://github.com/EfranH25/CS-674-Final-Project-3dInAction.git '/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction'

Cloning into '/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction'...
remote: Enumerating objects: 2911, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 2911 (delta 42), reused 64 (delta 25), pack-reused 2807
Receiving objects: 100% (2911/2911), 84.14 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (2031/2031), done.


Mount the Google Drive

In [ ]:
from google.colab import drive

help(drive)

from os import chdir as change_directory
from os import listdir as list_directory

# Mount my Google Drive at My Drive.
drive.mount('/content/drive')

Help on module google.colab.drive in google.colab:

NAME
    google.colab.drive - Colab-specific Google Drive integration.

FUNCTIONS
    flush_and_unmount(timeout_ms=86400000)
        Unmount Google Drive and flush any outstanding writes to it.
    
    mount(mountpoint, force_remount=False, timeout_ms=120000, readonly=False)
        Mount your Google Drive at the specified mountpoint path.

DATA
    __all__ = ['flush_and_unmount', 'mount']

FILE
    /usr/local/lib/python3.10/dist-packages/google/colab/drive.py


Mounted at /content/drive


# datasets/MSRAction3DDataset.py

In [ ]:
# Move in the cloned repository.
change_directory('/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/datasets')
list_directory('.')

['DfaustDataset.py',
 'IKEAActionDatasetClips.py',
 'IKEAEgoDatasetClips.py',
 '__init__.py',
 'MSRAction3DDataset.py']

In [ ]:
MSRAction3DDataset_py_content = "import torch\n\
import pickle\n\
import numpy as NumPy\n\
import torch.cuda as tcuda\n\
\n\
# If CUDA is available, run code on GPU. Else, run code on CPU.\n\
if tcuda.is_available():\n\
  this_device = torch.device('cuda:0')\n\
else:\n\
  this_device = torch.device('cpu')\n\
\n\
class MSRAction3DDataset(torch.utils.data.Dataset):\n\
  def __init__(self, dataset_path, set, cfg_data):\n\
    # Call the __init__ function of the Dataset superclass.\n\
    super(MSRAction3DDataset, self).__init__()\n\
\n\
    with open(dataset_path + 'MSRAction3D_FPS_Videos.pickle' , mode = 'rb') as msr_action3d_pickle_file:\n\
      msr_action3d_depth_map_sequences = pickle.load(msr_action3d_pickle_file)\n\
    msr_action3d_depth_map_sequence_labels = NumPy.load(dataset_path + 'MSRAction3D_FPS_Video_Labels.npz')['labels']\n\
\n\
    # Store the number of depth maps per clip in a variable.\n\
    clip_size = 8\n\
\n\
    # Initialize a dictionary to hold the MSRAction3D clips and their labels.\n\
    MSRAction3D_dataset = {'labels': [], 'clips': []}\n\
\n\
    # Loop through the depth map sequences of the MSRAction3D Dataset.\n\
    for index, depth_map_sequence in enumerate(msr_action3d_depth_map_sequences):\n\
      label = msr_action3d_depth_map_sequence_labels[index]\n\
      num_depth_maps = depth_map_sequence.shape[1]\n\
      clips = list(torch.Tensor(depth_map_sequence[0]).split(clip_size))\n\
      if num_depth_maps % clip_size != 0:\n\
        clips = clips[:-1]\n\
      MSRAction3D_dataset['labels'].append(label + torch.zeros((num_depth_maps // clip_size, )))\n\
      [MSRAction3D_dataset['clips'].append(clip[None]) for clip in clips]\n\
\n\
    # Concatenate all the label Tensors in the dataset together to form a single Tensor. Similarly, \n\
    # concatenate all the clip Tensors into a single Tensor.\n\
    labels = torch.concatenate(MSRAction3D_dataset['labels'])\n\
    subsequences = torch.concatenate(MSRAction3D_dataset['clips'])\n\
\n\
    # Calculate the portion of the dataset to use depending on whether\n\
    # the current phase is a training, validation or test phase.\n\
    if set == \"train\":\n\
      start_index = 0\n\
      end_index = int(0.6 * labels.shape[0])\n\
    elif set == \"test\":\n\
      start_index = int(0.6 * labels.shape[0])\n\
      end_index = labels.shape[0]\n\
\n\
    # Store the labels and subsequences in class variables.\n\
    self.labels = labels[start_index: end_index].to(device = this_device)\n\
    self.subsequences = subsequences[start_index: end_index].to(device = this_device)\n\
\n\
    # Store the clips in the dataset in a variable.\n\
    self.clip_set = MSRAction3D_dataset['clips']\n\
  def __getitem__(self, integral_key):\n\
    # Implemented the __getitem__ method of this dataset.\n\
    return (self.labels[integral_key], self.subsequences[integral_key])\n\
  def __len__(self):\n\
    # Implemented the __len__ method of this dataset.\n\
    return len(self.labels)\n\
  def __getitems__(self, batch_indices):\n\
    # Implemented the __getitems__ method of this dataset.\n\
    return (self.labels[batch_indices], self.subsequences[batch_indices])\n\
  def make_weights_for_balanced_classes(self):\n\
    label_counts = self.labels.unique(return_counts = True)[1]\n\
    self.num_classes = label_counts.shape[0]\n\
    return list(label_counts / label_counts.sum())"

with open('MSRAction3DDataset.py', 'w') as python_file:
  python_file.writelines(MSRAction3DDataset_py_content)

# datasets/__init__.py

In [ ]:
dataset_init_file = """
import torch
from torch.utils.data import DataLoader

# from .IKEAEgoDatasetClips import IKEAEgoDatasetClips
# from .DfaustDataset import DfaustActionClipsDataset
# from .IKEAActionDatasetClips import IKEAActionDatasetClips
from .MSRAction3DDataset import MSRAction3DDataset

import i3d_utils as utils

def build_dataset(cfg, training=True):
    split = 'train' if training else 'test'
    cfg_data = cfg['DATA']
    if cfg_data.get('name') == 'DFAUST':
        data_augmentation = cfg['TRAINING'].get('aug') if split == 'train' else cfg['TESTING'].get('aug')
        dataset = DfaustActionClipsDataset(
            action_dataset_path=cfg_data['dataset_path'], frames_per_clip=cfg_data['frames_per_clip'], set=split,
            n_points=cfg_data['n_points'], shuffle_points=cfg_data['shuffle_points'], gender=cfg_data['gender'],
            data_augmentation=data_augmentation, noisy_data=cfg_data['noisy_data'],
        )
    elif cfg_data.get('name') == 'IKEA_ASM':
        dataset = IKEAActionDatasetClips(dataset_path=cfg_data['dataset_path'], set=split)
    elif cfg_data.get('name') == 'IKEA_EGO':
        dataset = IKEAEgoDatasetClips(dataset_path=cfg_data['dataset_path'], set=split, cfg_data=cfg_data)
    elif cfg_data.get('name') == 'MSR-Action3D':
        dataset = MSRAction3DDataset(dataset_path=cfg_data['dataset_path'], set=split, cfg_data=cfg_data)
    else:
        raise NotImplementedError
    return dataset


def build_dataloader(config, training=True, shuffle=False):
    dataset = build_dataset(config, training)

    num_workers = config['num_workers']
    batch_size = config['TRAINING'].get('batch_size') if training else config['TESTING'].get('batch_size')
    data_sampler = config['DATA'].get('data_sampler')

    split = 'train' if training else 'test'
    print("Number of clips in the {} set:{}".format(split, len(dataset)))

    if training and data_sampler == 'weighted':
        if config['DATA'].get('name') == 'DFAUST':
            weights = dataset.make_weights_for_balanced_classes()
        elif config['DATA'].get('name') == 'IKEA_ASM' or config['DATA'].get('name') == 'IKEA_EGO':
            weights = utils.make_weights_for_balanced_classes(dataset.clip_set, dataset.clip_label_count)
        elif config['DATA'].get('name') == 'MSR-Action3D':
            weights = dataset.make_weights_for_balanced_classes()
        else:
            raise NotImplementedError
        sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
    else:
        sampler = None

    dataloader = DataLoader(
        dataset=dataset,
        shuffle=shuffle,
        num_workers = num_workers,
        sampler = sampler,
        collate_fn = lambda x: x,
        batch_size=batch_size
    )
    return dataloader, dataset
"""

with open('__init__.py', 'w') as python_file:
  python_file.writelines(dataset_init_file)

# config/msr-action3d/config_msr_action3d.yaml

In [ ]:
import yaml
from os import getcwd as pwd
from os import makedirs as newdir
from os.path import exists as file_existing

#help(yaml)

yaml_document = None

newdir('../configs/msr-action3d', exist_ok = True)

if file_existing('/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/MSRAction3DFPS/configs/msr-action3d/config_msr_action3d.yaml'):
  with open('/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/MSRAction3DFPS/configs/msr-action3d/config_msr_action3d.yaml', 'r') as yaml_file:
    yaml_document = yaml.safe_load( yaml_file)
else:
  with open('../configs/dfaust/config_dfaust.yaml', 'r') as yaml_file:
    yaml_document = yaml.safe_load(yaml_file)
    yaml_document['DATA']['name'] = 'MSR-Action3D'
    yaml_document['DATA']['dataset_path'] = '/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/MSRAction3DFPS/MSRAction3D_fps/'
    yaml_document['MODEL']['pc_model'] = 'set_transformer'
    yaml_document['TRAINING']['batch_size'] = 32
    yaml_document['TRAINING']['n_epochs'] = 1800
    yaml_document['TRAINING']['refine'] = True
    yaml_document['TRAINING']['refine_epoch'] = 1500
    yaml_document['TRAINING']['lr'] = 1e-3

with open('../configs/msr-action3d/config_msr_action3d.yaml', 'w') as yaml_file:
  yaml.safe_dump(yaml_document, yaml_file)




# Load Wandb files.

In [ ]:
from shutil import copy as copy_file

file_1 = "/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/MSRAction3DFPS/log/debug/{:06d}.pt".format(yaml_document['TRAINING']['refine_epoch'])
file_2 = "/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/log/debug/{:06d}.pt".format(yaml_document['TRAINING']['refine_epoch'])

newdir('/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/log/debug/', exist_ok = True)
copy_file(file_1, file_2)

'/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/log/debug/002700.pt'

# models/pointnet2_utils.py

In [ ]:
pointnet2_utils_py = """
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time
import numpy as np
from torch.autograd import Function
import warnings

# try:
#     import models._ext as _ext
# except ImportError:
#     from torch.utils.cpp_extension import load
#     import glob
#     import os.path as osp
#     import os

#     warnings.warn("Unable to load pointnet2_ops cpp extension. JIT Compiling.")

#     _ext_src_root = osp.join(osp.dirname(__file__), "_ext-src")
#     _ext_sources = glob.glob(osp.join(_ext_src_root, "src", "*.cpp")) + glob.glob(
#         osp.join(_ext_src_root, "src", "*.cu")
#     )
#     _ext_headers = glob.glob(osp.join(_ext_src_root, "include", "*"))

#     os.environ["TORCH_CUDA_ARCH_LIST"] = "3.7+PTX;5.0;6.0;6.1;6.2;7.0;7.5"
#     _ext = load(
#         "_ext",
#         sources=_ext_sources,
#         extra_include_paths=[osp.join(_ext_src_root, "include")],
#         extra_cflags=["-O3"],
#         extra_cuda_cflags=["-O3", "-Xfatbin", "-compress-all"],
#         with_cuda=True,
#     )

def timeit(tag, t):
    print("{}: {}s".format(tag, time() - t))
    return time()

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

def square_distance(src, dst):
    \"\"\"
    Calculate Euclid distance between each two points.

    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst

    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    \"\"\"
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist


def index_points(points, idx):
    \"\"\"

    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    \"\"\"
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    idx = idx.long()
    new_points = points[batch_indices, idx, :]
    return new_points


# This implementation is very slow and should not be used unless you can't get the CUDA version to compile
# def farthest_point_sample(xyz, npoint):
#     \"\"\"
#     Input:
#         xyz: pointcloud data, [B, N, 3]
#         npoint: number of samples
#     Return:
#         centroids: sampled pointcloud index, [B, npoint]
#     \"\"\"
#     device = xyz.device
#     B, N, C = xyz.shape
#     centroids = torch.zeros(B, npoint, dtype=torch.long, device=device)
#     distance = torch.ones(B, N, device=device)* 1e10
#     farthest = torch.randint(0, N, (B,), dtype=torch.long, device=device)
#     batch_indices = torch.arange(B, dtype=torch.long, device=device)
#     for i in range(npoint):
#         centroids[:, i] = farthest
#         centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
#         dist = torch.sum((xyz - centroid) ** 2, -1)
#         mask = dist < distance
#         distance[mask] = dist[mask]
#         farthest = torch.max(distance, -1)[1]
#     return centroids


def query_ball_point(radius, nsample, xyz, new_xyz):
    \"\"\"
    Input:
        radius: local region radius
        nsample: max sample number in local region
        xyz: all points, [B, N, 3]
        new_xyz: query points, [B, S, 3]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    \"\"\"
    device = xyz.device
    B, N, C = xyz.shape
    _, S, _ = new_xyz.shape
    group_idx = torch.arange(N, dtype=torch.long).to(device).view(1, 1, N).repeat([B, S, 1])
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :nsample]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat([1, 1, nsample])
    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    return group_idx


def sample_and_group(npoint, radius, nsample, xyz, points, returnfps=False):
    \"\"\"
    Input:
        npoint:
        radius:
        nsample:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, npoint, nsample, 3]
        new_points: sampled points data, [B, npoint, nsample, 3+D]
    \"\"\"
    B, N, C = xyz.shape
    S = npoint
    fps_idx = farthest_point_sample(xyz, npoint) # [B, npoint, C]
    new_xyz = index_points(xyz, fps_idx.to(torch.int64))

    idx = query_ball_point(radius, nsample, xyz, new_xyz)
    # dists = square_distance(new_xyz, xyz)  # B x npoint x N
    # idx = dists.argsort()[:, :, :nsample]

    grouped_xyz = index_points(xyz, idx) # [B, npoint, nsample, C]
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, C)

    if points is not None:
        grouped_points = index_points(points, idx)
        new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1) # [B, npoint, nsample, C+D]
    else:
        new_points = grouped_xyz_norm
    if returnfps:
        return new_xyz, new_points, grouped_xyz, fps_idx
    else:
        return new_xyz, new_points


def sample_and_group_all(xyz, points):
    \"\"\"
    Input:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, 1, 3]
        new_points: sampled points data, [B, 1, N, 3+D]
    \"\"\"
    device = xyz.device
    B, N, C = xyz.shape
    new_xyz = torch.zeros(B, 1, C).to(device)
    grouped_xyz = xyz.view(B, 1, N, C)
    if points is not None:
        new_points = torch.cat([grouped_xyz, points.view(B, 1, N, -1)], dim=-1)
    else:
        new_points = grouped_xyz
    return new_xyz, new_points


def sample_and_group_4d(npoint, radius, nsample, xyz, points, returnfps=False):
    \"\"\"
    Input:
        npoint:
        radius:
        nsample:
        xyz: input points position data, [B, t, N, 3]
        points: input points data, [B, t, N, D]
    Return:
        new_xyz: sampled points position data, [B, t, npoint, nsample, 3]
        new_points: sampled points data, [B, t, npoint, nsample, 3+D]
    \"\"\"
    B, t, N, C = xyz.shape
    S = npoint
    D = points.shape[-1]
    fps_idx = farthest_point_sample(xyz[:, 0], npoint)  # [B, npoint, C], use frist drame for fps
    xyz = xyz.reshape(-1, N, C)
    new_xyz = index_points(xyz, fps_idx.unsqueeze(1).repeat([1, t, 1]).reshape(-1, npoint))
    idx = query_ball_point(radius, nsample, xyz, new_xyz)
    grouped_xyz = index_points(xyz, idx)  # [B, npoint, nsample, C]
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B*t, S, 1, C)  # shouldnt this also be scaled to unit sphere?

    if points is not None:
        grouped_points = index_points(points.reshape(-1, N, D), idx)
        new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1)  # [B, npoint, nsample, C+D]
    else:
        new_points = grouped_xyz_norm

    new_xyz = new_xyz.reshape(B, t, npoint, C)
    new_points = new_points.reshape(B, t, npoint, nsample, -1)

    if returnfps:
        return new_xyz, new_points, grouped_xyz, fps_idx
    else:
        return new_xyz, new_points

def sample_and_group_all_4d(xyz, points):
    \"\"\"
    Input:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, 1, 3]
        new_points: sampled points data, [B, 1, N, 3+D]
    \"\"\"
    device = xyz.device
    B, t, N, C = xyz.shape
    new_xyz = torch.zeros(B, t, 1, C).to(device)
    grouped_xyz = xyz.view(B, t, 1, N, C)
    if points is not None:
        new_points = torch.cat([grouped_xyz, points.view(B, t, 1, N, -1)], dim=-1)
    else:
        new_points = grouped_xyz
    return new_xyz, new_points


class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all):
        super(PointNetSetAbstraction, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv3d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm3d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        \"\"\"
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        \"\"\"
        b, t, k, n = xyz.shape
        xyz = xyz.reshape(-1, k, n)
        xyz = xyz.permute(0, 2, 1).contiguous()
        if points is not None:
            points = points.permute(0, 1, 3, 2)
            points = points.reshape(-1, points.shape[-2], points.shape[-1])

        if self.group_all:
            new_xyz, new_points = sample_and_group_all(xyz, points)
        else:
            new_xyz, new_points = sample_and_group(self.npoint, self.radius, self.nsample, xyz, points)
        # new_xyz: sampled points position data, [b*t, npoint, k]
        # new_points: sampled points data, [b*t, npoint, nsample, d+k]
        new_points = new_points.reshape(b, t, new_points.shape[-3], new_points.shape[-2], new_points.shape[-1])
        new_points = new_points.permute(0, 4, 2, 1, 3)  # [b, d+k, npoint, t, nsample]
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))

        new_points = torch.max(new_points, -1)[0]
        new_points = new_points.permute(0, 3, 1, 2)
        new_xyz = new_xyz.reshape(b, t, new_xyz.shape[-2], new_xyz.shape[-1]).permute(0, 1, 3, 2)
        return new_xyz, new_points

class PointNetPP4DSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all, temporal_conv=4):
        super(PointNetPP4DSetAbstraction, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv3d(last_channel, out_channel, [1, temporal_conv, 1], 1, padding='same'))
            self.mlp_bns.append(nn.BatchNorm3d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        \"\"\"
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        \"\"\"
        b, t, k, n = xyz.shape
        xyz = xyz.reshape(-1, k, n)
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 1, 3, 2)
            # points = points.reshape(-1, points.shape[-2], points.shape[-1])

        if self.group_all:
            new_xyz, new_points = sample_and_group_all_4d(xyz.reshape(b, t, n, k), points)
        else:
            new_xyz, new_points = sample_and_group_4d(self.npoint, self.radius, self.nsample,
                                                      xyz.reshape(b, t, n, k), points)

        # new_xyz: sampled points position data, [b*t, npoint, k]
        # new_points: sampled points data, [b*t, npoint, nsample, d+k]
        new_points = new_points.reshape(b, t, new_points.shape[-3], new_points.shape[-2], new_points.shape[-1])
        new_points = new_points.permute(0, 4, 2, 1, 3)  # [b, d+k, npoint, t, nsample]
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))

        new_points = torch.max(new_points, -1)[0]
        new_points = new_points.permute(0, 3, 1, 2)
        new_xyz = new_xyz.reshape(b, t, new_xyz.shape[-2], new_xyz.shape[-1]).permute(0, 1, 3, 2)
        return new_xyz, new_points

class PointNetSetAbstractionMsg(nn.Module):
    def __init__(self, npoint, radius_list, nsample_list, in_channel, mlp_list):
        super(PointNetSetAbstractionMsg, self).__init__()
        self.npoint = npoint
        self.radius_list = radius_list
        self.nsample_list = nsample_list
        self.conv_blocks = nn.ModuleList()
        self.bn_blocks = nn.ModuleList()
        for i in range(len(mlp_list)):
            convs = nn.ModuleList()
            bns = nn.ModuleList()
            last_channel = in_channel # + 3
            for out_channel in mlp_list[i]:
                convs.append(nn.Conv3d(last_channel, out_channel, 1))
                bns.append(nn.BatchNorm3d(out_channel))
                last_channel = out_channel
            self.conv_blocks.append(convs)
            self.bn_blocks.append(bns)

    def forward(self, xyz, points):
        \"\"\"
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        \"\"\"
        b, t, k, n = xyz.shape
        xyz = xyz.reshape(-1, k, n)
        xyz = xyz.permute(0, 2, 1).contiguous()
        if points is not None:
            points = points.permute(0, 1, 3, 2)
            points = points.reshape(-1, points.shape[-2], points.shape[-1])

        B, N, C = xyz.shape
        S = self.npoint
        new_xyz = index_points(xyz, farthest_point_sample(xyz, S))
        new_points_list = []
        for i, radius in enumerate(self.radius_list):
            K = self.nsample_list[i]
            group_idx = query_ball_point(radius, K, xyz, new_xyz)
            grouped_xyz = index_points(xyz, group_idx)
            grouped_xyz -= new_xyz.view(B, S, 1, C)
            if points is not None:
                grouped_points = index_points(points, group_idx)
                grouped_points = torch.cat([grouped_points, grouped_xyz], dim=-1)
            else:
                grouped_points = grouped_xyz

            grouped_points = grouped_points.reshape(b, t, grouped_points.shape[-3], grouped_points.shape[-2], grouped_points.shape[-1])
            grouped_points = grouped_points.permute(0, 4, 2, 1, 3)  # [b, d+k, npoint, t, nsample]

            for j in range(len(self.conv_blocks[i])):
                conv = self.conv_blocks[i][j]
                bn = self.bn_blocks[i][j]
                grouped_points = F.relu(bn(conv(grouped_points)))
            new_points = torch.max(grouped_points, -1)[0]
            new_points_list.append(new_points)

        new_xyz = new_xyz.reshape(b, t, new_xyz.shape[-2], new_xyz.shape[-1]).permute(0, 1, 3, 2)
        new_points_concat = torch.cat(new_points_list, dim=1)
        new_points_concat = new_points_concat.permute(0, 3, 1, 2)
        return new_xyz, new_points_concat


class PointNetFeaturePropagation(nn.Module):
    def __init__(self, in_channel, mlp):
        super(PointNetFeaturePropagation, self).__init__()
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv1d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm1d(out_channel))
            last_channel = out_channel

    def forward(self, xyz1, xyz2, points1, points2):
        \"\"\"
        Input:
            xyz1: input points position data, [B, C, N]
            xyz2: sampled input points position data, [B, C, S]
            points1: input points data, [B, D, N]
            points2: input points data, [B, D, S]
        Return:
            new_points: upsampled points data, [B, D', N]
        \"\"\"
        xyz1 = xyz1.permute(0, 2, 1)
        xyz2 = xyz2.permute(0, 2, 1)

        points2 = points2.permute(0, 2, 1)
        B, N, C = xyz1.shape
        _, S, _ = xyz2.shape

        if S == 1:
            interpolated_points = points2.repeat(1, N, 1)
        else:
            dists = square_distance(xyz1, xyz2)
            dists, idx = dists.sort(dim=-1)
            dists, idx = dists[:, :, :3], idx[:, :, :3]  # [B, N, 3]

            dist_recip = 1.0 / (dists + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm
            interpolated_points = torch.sum(index_points(points2, idx) * weight.view(B, N, 3, 1), dim=2)

        if points1 is not None:
            points1 = points1.permute(0, 2, 1)
            new_points = torch.cat([points1, interpolated_points], dim=-1)
        else:
            new_points = interpolated_points

        new_points = new_points.permute(0, 2, 1)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        return new_points


class PointNetSetAbstractionOriginal(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all, knn=False):
        super(PointNetSetAbstractionOriginal, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.knn = knn
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        \"\"\"
        Input:
            xyz: input points position data, [B, N, C]
            points: input points data, [B, N, C]
        Return:
            new_xyz: sampled points position data, [B, S, C]
            new_points_concat: sample points feature data, [B, S, D']
        \"\"\"
        xyz = xyz.permute(0, 2, 1)
        if self.group_all:
            new_xyz, new_points = sample_and_group_all(xyz, points)
        else:
            new_xyz, new_points = sample_and_group(self.npoint, self.radius, self.nsample, xyz, points)
        # new_xyz: sampled points position data, [B, npoint, C]
        # new_points: sampled points data, [B, npoint, nsample, C+D]
        new_points = new_points.permute(0, 3, 2, 1) # [B, C+D, nsample,npoint]
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points =  F.relu(bn(conv(new_points)))

        new_points = torch.max(new_points, 2)[0].transpose(1, 2)
        return new_xyz, new_points


class FurthestPointSampling(Function):
    @staticmethod
    def forward(ctx, xyz, npoint):
        # type: (Any, torch.Tensor, int) -> torch.Tensor
        r\"\"\"
        Uses iterative furthest point sampling to select a set of npoint features that have the largest
        minimum distance
        Parameters
        ----------
        xyz : torch.Tensor
            (B, N, 3) tensor where N > npoint
        npoint : int32
            number of features in the sampled set
        Returns
        -------
        torch.Tensor
            (B, npoint) tensor containing the set
        \"\"\"
        fps_inds = _ext.furthest_point_sampling(xyz, npoint)
        ctx.mark_non_differentiable(fps_inds)
        return fps_inds

    @staticmethod
    def backward(xyz, a=None):
        return None, None


farthest_point_sample = FurthestPointSampling.apply
"""

with open('../models/pointnet2_utils.py', 'w') as python_file:
  python_file.writelines(pointnet2_utils_py)

# models/__init__.py

In [ ]:
change_directory('/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/datasets')

In [ ]:
models_init_py = """
import os
import importlib
import sys

print("DEBUG")

from .pointnet import PointNet1, PointNet1Basic

print("DEBUG")

# from .pointnet2_cls_ssg import PointNet2, PointNet2Basic

print("DEBUG")

from .pytorch_3dmfv import FourDmFVNet

print("DEBUG")

#from .tpatches import TPatchesInAction
from .set_transformer import SetTransformerTemporal

print("DEBUG-22")

#from .tpatch_trajectory import tPatchTraj
from .DGCNN import DGCNN

print("DEBUG-27")

#from .pstnet import PSTnet

print("DEBUG-31")

# from .PST_Transformer import PSTTransformer
# from .P4Transformer import P4Transformer

print("DEBUG")

__all__ = {
    'pn1': PointNet1,
    'pn1_4d_basic': PointNet1Basic,
    #'pn2': PointNet2,
    #'pn2_4d_basic': PointNet2Basic,
    #'tpatches': TPatchesInAction,
    '3dmfv': FourDmFVNet,
    'set_transformer': SetTransformerTemporal,
    #'tpatch_trajectory': tPatchTraj,
    'dgcnn': DGCNN,
#    'pst_transformer': PSTTransformer,
#    'pstnet': PSTnet,
#    'p4transformer': P4Transformer,
}

def build_model(model_cfg, num_class, frames_per_clip):
    model = __all__[model_cfg['pc_model']](
        model_cfg=model_cfg, num_class=num_class, n_frames=frames_per_clip
    )
    return model

file_name_dict = {
    'pn1': "pointnet.py",
    'pn1_4d_basic': "pointnet.py",
    'pn2': "pointnet2_cls_ssg.py",
    'pn2_4d_basic': "pointnet2_cls_ssg.py",
    'tpatches': "tpatches.py",
    '3dmfv': "pytorch_3dmfv.py",
    'set_transformer': 'set_transformer.py',
    'tpatch_trajectory': 'tpatch_trajectory.py',
    'dgcnn': 'DGCNN.py',
    'pstnet': 'pstnet.py',
    'pst_transformer': 'PST_Transformer.py',
    'p4transformer': 'P4Transformer.py',
}


class build_model_from_logdir(object):
    def __init__(self, logdir, model_cfg, num_classes, frames_per_clip):
        pc_model = model_cfg.get('pc_model')
        model_instance = __all__[pc_model]
        model_name = model_instance.__name__
        file_name = file_name_dict.get(pc_model)

        spec = importlib.util.spec_from_file_location(model_name, os.path.join(logdir, 'models', file_name))
        import_model = importlib.util.module_from_spec(spec)
        sys.modules[model_name] = import_model
        spec.loader.exec_module(import_model)
        self.model = model_instance(model_cfg=model_cfg, num_class=num_classes, n_frames=frames_per_clip)
    def get(self):
        return self.model

"""

with open('../models/__init__.py', 'w') as python_file:
  python_file.writelines(models_init_py)

# models/extractors.py

In [ ]:
extractors_py = """
import faiss
from faiss.contrib.torch_utils import torch_replacement_knn_gpu as faiss_torch_knn_gpu
from pykeops.torch import Vi, Vj
from scipy.spatial import cKDTree
import numpy as np
import torch
import torch.nn as nn
import models.pointnet2_utils as utils


def get_tpatches(x1, x2, feat_seq, flip=False, k=16, radius=0.2, res=None,
                 sample_mode='nn', add_centroid_jitter=None, downsample_method='fps', npoints=None):
    b, t, n, d = x1.shape
    n_out = n

    if feat_seq is None:
        feat_seq = x1
        d_feat = d
    else:
        d_feat = feat_seq.shape[-1]

    out_x = torch.empty(b, t, n_out, d)
    patchlets = torch.empty(b, t, n, k, device=x1.device, dtype=torch.long)
    distances_i = torch.empty(b, t, n, k, device=x1.device)
    idxs_i = torch.empty(b, t, n, k, device=x1.device, dtype=torch.long)
    patchlet_points = torch.empty(b, t, n, k, d, device=x1.device)
    patchlet_feats = torch.empty(b, t, n, k, d_feat, device=x1.device)

    # loop over the data to reorder the indices to form the patchlets
    x_current = x2[:, 0]
    feat_seq_2 = torch.cat([feat_seq[:, [0]], feat_seq], dim=1)[:, :-1]
    for i in range(0, t):
        x_next = x1[:, i]
        distances, idxs = get_knn(x_current[..., 0:3], x_next[..., 0:3], k=k, res=res, method='keops',
                                  radius=radius)
        if sample_mode == 'nn':
            x_current = utils.index_points(x_next, idxs)[:, :, 0, :]
        elif sample_mode == 'randn':
            rand_idx = torch.randint(k, (b, n, 1), device=x_next.device, dtype=torch.int64).repeat(
                [1, 1, 3]).unsqueeze(2)
            x_current = torch.gather(utils.index_points(x_next, idxs).squeeze(), dim=2, index=rand_idx).squeeze()
        elif sample_mode == 'gt':
            # only works when point correspondence is known and points are already aligned
            x_current = x_next
        else:
            raise ValueError("sample mode not supported")

        out_x[:, i] = x_current
        if add_centroid_jitter is not None and not sample_mode == 'gt':
            x_current = x_current + add_centroid_jitter * torch.randn_like(x_current)

        distances_i[:, i], idxs_i[:, i] = distances, idxs
        patchlets[:, i] = idxs_i[:, i]
        patchlet_points[:, i] = utils.index_points(x_next, idxs).squeeze()
        patchlet_feats[:, i] = utils.index_points(feat_seq_2[:, i], idxs).squeeze()

    distances = distances_i
    idxs = idxs_i
    if flip:  # reverse temporal order
        patchlet_feats = torch.flip(patchlet_feats, [1])
        patchlet_points = torch.flip(patchlet_points, [1])
        idxs = torch.flip(idxs, [1])
        distances = torch.flip(distances, [1])
        patchlets = torch.flip(patchlets, [1])

    patchlet_feats = patchlet_feats.reshape(b * t, n, k, d_feat)
    patchlet_points = patchlet_points.reshape(b * t, n, k, d)
    idxs = idxs.reshape(b * t, n, k)
    distances = distances.reshape(b * t, n, k)
    patchlets = patchlets.reshape(b * t, n, k)

    fps_idx = None
    if downsample_method == 'fps':
        # select a subset of the points using fps for maximum coverage
        selected_idxs = utils.farthest_point_sample(x1[:, 0].contiguous(), npoints).to(torch.int64)
        selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, npoints)
        patchlet_points = utils.index_points(patchlet_points, selected_idxs)
        patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
        distances = utils.index_points(distances, selected_idxs)
        idxs = utils.index_points(idxs, selected_idxs)
        patchlets = utils.index_points(patchlets, selected_idxs)
        n = npoints
    elif downsample_method == 'var' or downsample_method == 'mean_var_t':
        # select a subset of the points with the largest point variance for maximum temporal movement
        if downsample_method == 'var':
            temporal_patchlet_points = patchlet_points.reshape(b, t, n, k, d). \
                permute(0, 2, 1, 3, 4).reshape(b, n, -1, d)
            patchlet_variance = torch.linalg.norm(torch.var(temporal_patchlet_points, -2), dim=-1)
        elif downsample_method == 'mean_var_t':
            patchlet_variance = torch.linalg.norm(
                torch.var(torch.mean(patchlet_points.reshape(b, t, n, k, d), -2), 1), dim=-1)
        else:
            raise ValueError("downsample method not supported ")
        _, selected_idxs = torch.topk(patchlet_variance, npoints)
        selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, npoints)
        patchlet_points = utils.index_points(patchlet_points, selected_idxs)
        patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
        distances = utils.index_points(distances, selected_idxs)
        idxs = utils.index_points(idxs, selected_idxs)
        patchlets = utils.index_points(patchlets, selected_idxs)
        n = npoints

    return patchlet_points, patchlet_feats, distances, idxs, patchlets, n, d_feat, fps_idx, out_x


def get_knn(x1, x2, k=16, res=None, method='faiss_gpu', radius=None):

    if method == 'faiss_gpu':
        distances, idxs = faiss_torch_knn_gpu(res, x1, x2, k=k)
    if method == 'faiss_cpu':
        distances, idxs = faiss.knn(np.ascontiguousarray(x1.detach().cpu().numpy(), dtype=np.float32),
                                    np.ascontiguousarray(x2.detach().cpu().numpy(), dtype=np.float32), k=k)
        distances, idxs = torch.tensor(distances, device=x1.device).cuda(), torch.tensor(idxs, device=x1.device).cuda()
    if method == 'spatial':
        tree = cKDTree(x2.detach().cpu().numpy())
        distances, idxs = tree.query(x1.detach().cpu().numpy(), k, workers=-1)
        distances, idxs = torch.tensor(distances).cuda(), torch.tensor(idxs).cuda()
    if method == 'keops': #supports batch operaations
        X_i = Vi(0, x1.shape[-1])
        X_j = Vj(1, x2.shape[-1])
        D_ij = ((X_i - X_j) ** 2).sum(-1)
        KNN_fun = D_ij.Kmin_argKmin(k, dim=1)
        distances, idxs = KNN_fun(x1.contiguous(), x2.contiguous())

    if radius is not None:
        # clip samples outside a radius, replace with origin to keep k constant
        clipped_idxs = idxs[..., [0]].repeat(1, 1, k)
        mask = distances > radius**2
        idxs[mask] = clipped_idxs[mask]
        distances[mask] = 0
    return distances, idxs


class TPatchExtractor(nn.Module):
    def __init__(self, k=16, sample_mode='nn', npoints=None, add_centroid_jitter=None, downsample_method=None,
                 radius=None, temporal_stride=8):
        super(TPatchExtractor, self).__init__()
        self.k = k
        self.radius = radius
        self.sample_mode = sample_mode
        self.downsample_method = downsample_method
        self.npoints = npoints
        self.add_centroid_jitter = add_centroid_jitter
        self.res = faiss.StandardGpuResources()
        self.res.setDefaultNullStreamAllDevices()


    def forward(self, point_seq, feat_seq=None):
        b, t, n, d = point_seq.shape
        n_original = n
        n_out = n
        if feat_seq is None:
            feat_seq = point_seq
            d_feat = d
        else:
            d_feat = feat_seq.shape[-1]

        x1 = point_seq
        x2 = torch.cat([point_seq[:, [0]], point_seq], dim=1)[:, :-1]

        out_x = torch.empty(b, t, n_out, d)
        patchlets = torch.empty(b, t, n, self.k, device=point_seq.device, dtype=torch.long)
        distances_i = torch.empty(b,  t, n, self.k, device=point_seq.device)
        idxs_i = torch.empty(b, t, n, self.k, device=point_seq.device, dtype=torch.long)
        patchlet_points = torch.empty(b, t, n, self.k, d, device=point_seq.device)
        patchlet_feats = torch.empty(b, t, n, self.k, d_feat, device=point_seq.device)

        # loop over the data to reorder the indices to form the patchlets
        x_current = x2[:, 0]
        feat_seq_2 = torch.cat([feat_seq[:, [0]], feat_seq], dim=1)[:, :-1]
        for i in range(0, t):
            x_next = x1[:, i]
            distances, idxs = get_knn(x_current[..., 0:3], x_next[..., 0:3], k=self.k, res=self.res, method='keops', radius=self.radius)
            if self.sample_mode == 'nn':
                x_current = utils.index_points(x_next, idxs)[:, :, 0, :]
            elif self.sample_mode == 'randn':
                rand_idx = torch.randint(self.k, (b, n, 1), device=x_next.device, dtype=torch.int64).repeat([1, 1, 3]).unsqueeze(2)
                x_current = torch.gather(utils.index_points(x_next, idxs).squeeze(), dim=2, index=rand_idx).squeeze()
            elif self.sample_mode == 'gt':
                # only works when point correspondence is known and points are already aligned
                x_current = x_next
            else:
                raise ValueError("sample mode not supported")

            out_x[:, i] = x_current
            if self.add_centroid_jitter is not None and not self.sample_mode == 'gt':
                x_current = x_current + self.add_centroid_jitter*torch.randn_like(x_current)

            distances_i[:, i], idxs_i[:, i] = distances, idxs
            patchlets[:, i] = idxs_i[:, i]
            patchlet_points[:, i] = utils.index_points(x_next, idxs).squeeze()
            patchlet_feats[:, i] = utils.index_points(feat_seq_2[:, i], idxs).squeeze()

        distances = distances_i
        idxs = idxs_i

        patchlet_feats = patchlet_feats.reshape(b*t, n, self.k, d_feat)
        patchlet_points = patchlet_points.reshape(b * t, n, self.k, d)
        idxs = idxs.reshape(b*t, n, self.k)
        distances = distances.reshape(b*t, n, self.k)
        patchlets = patchlets.reshape(b*t, n, self.k)

        fps_idx = None
        if self.downsample_method == 'fps':
            #select a subset of the points using fps for maximum coverage
            selected_idxs = utils.farthest_point_sample(point_seq[:, 0].contiguous(), self.npoints).to(torch.int64)
            selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, self.npoints)
            patchlet_points = utils.index_points(patchlet_points, selected_idxs)
            patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
            distances = utils.index_points(distances, selected_idxs)
            idxs = utils.index_points(idxs, selected_idxs)
            patchlets = utils.index_points(patchlets, selected_idxs)
            n = self.npoints
        elif self.downsample_method == 'var' or self.downsample_method == 'mean_var_t':
            # select a subset of the points with the largest point variance for maximum temporal movement
            if self.downsample_method == 'var':
                temporal_patchlet_points = patchlet_points.reshape(b, t, n, self.k, d).permute(0, 2, 1, 3, 4).reshape(b,n,-1,d)
                patchlet_variance = torch.linalg.norm(torch.var(temporal_patchlet_points, -2), dim=-1)
            elif self.downsample_method == 'mean_var_t':
                patchlet_variance = torch.linalg.norm(torch.var(torch.mean(patchlet_points.reshape(b, t, n, self.k, d), -2), 1), dim=-1)
            else:
                raise ValueError("downsample method not supported ")
            _, selected_idxs = torch.topk(patchlet_variance, self.npoints)
            selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, self.npoints)
            patchlet_points = utils.index_points(patchlet_points, selected_idxs)
            patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
            distances = utils.index_points(distances, selected_idxs)
            idxs = utils.index_points(idxs, selected_idxs)
            patchlets = utils.index_points(patchlets, selected_idxs)
            n = self.npoints


        # reshape all to bxtxnxk
        distances, idxs = distances.reshape(b, t, n, self.k), idxs.reshape(b, t, n, self.k)
        patchlets, patchlet_points = patchlets.reshape(b, t, n, self.k), patchlet_points.reshape(b, t, n, self.k, d)
        patchlet_feats = patchlet_feats.reshape(b, t, n, self.k, d_feat)

        normalized_patchlet_points = patchlet_points - patchlet_points[:, 0, :, [0], :].unsqueeze(1).detach() # normalize the patchlet around the center point of the first frame
        patchlet_feats = torch.cat([patchlet_feats, normalized_patchlet_points], -1)

        return {'idx': idxs, 'distances': distances, 'patchlets': patchlets,
                'patchlet_points': patchlet_points, 'patchlet_feats': patchlet_feats,
                'normalized_patchlet_points': normalized_patchlet_points, 'fps_idx': fps_idx,
                'x_current': out_x.reshape(b, t, n_out, d)}



class TPatchExtractorBidirectional(nn.Module):
    def __init__(self, k=16, sample_mode='nn', npoints=None, add_centroid_jitter=None, downsample_method=None,
                 radius=None, temporal_stride=8):
        super(TPatchExtractorBidirectional, self).__init__()
        self.k = k
        self.radius = radius
        self.sample_mode = sample_mode
        self.downsample_method = downsample_method
        self.npoints = npoints
        self.add_centroid_jitter = add_centroid_jitter
        self.res = faiss.StandardGpuResources()
        self.res.setDefaultNullStreamAllDevices()

    # def get_tpatches(self, x1, x2, feat_seq, flip=False):
    #
    #     b, t, n, d = x1.shape
    #     n_out = n
    #
    #     if feat_seq is None:
    #         feat_seq = x1
    #         d_feat = d
    #     else:
    #         d_feat = feat_seq.shape[-1]
    #
    #     out_x = torch.empty(b, t, n_out, d)
    #     patchlets = torch.empty(b, t, n, self.k, device=x1.device, dtype=torch.long)
    #     distances_i = torch.empty(b, t, n, self.k, device=x1.device)
    #     idxs_i = torch.empty(b, t, n, self.k, device=x1.device, dtype=torch.long)
    #     patchlet_points = torch.empty(b, t, n, self.k, d, device=x1.device)
    #     patchlet_feats = torch.empty(b, t, n, self.k, d_feat, device=x1.device)
    #
    #     # loop over the data to reorder the indices to form the patchlets
    #     x_current = x2[:, 0]
    #     feat_seq_2 = torch.cat([feat_seq[:, [0]], feat_seq], dim=1)[:, :-1]
    #     for i in range(0, t):
    #         x_next = x1[:, i]
    #         distances, idxs = get_knn(x_current[..., 0:3], x_next[..., 0:3], k=self.k, res=self.res, method='keops',
    #                                   radius=self.radius)
    #         if self.sample_mode == 'nn':
    #             x_current = utils.index_points(x_next, idxs)[:, :, 0, :]
    #         elif self.sample_mode == 'randn':
    #             rand_idx = torch.randint(self.k, (b, n, 1), device=x_next.device, dtype=torch.int64).repeat(
    #                 [1, 1, 3]).unsqueeze(2)
    #             x_current = torch.gather(utils.index_points(x_next, idxs).squeeze(), dim=2, index=rand_idx).squeeze()
    #         elif self.sample_mode == 'gt':
    #             # only works when point correspondence is known and points are already aligned
    #             # if self.downsample_method == 'fps':
    #             #     x_current = utils.index_points(x_next, fps_idx).contiguous()
    #             # else:
    #             x_current = x_next
    #         else:
    #             raise ValueError("sample mode not supported")
    #
    #         out_x[:, i] = x_current
    #         if self.add_centroid_jitter is not None  and not self.sample_mode == 'gt':
    #             x_current = x_current + self.add_centroid_jitter * torch.randn_like(x_current)
    #
    #         distances_i[:, i], idxs_i[:, i] = distances, idxs
    #         patchlets[:, i] = idxs_i[:, i]
    #         patchlet_points[:, i] = utils.index_points(x_next, idxs).squeeze()
    #         patchlet_feats[:, i] = utils.index_points(feat_seq_2[:, i], idxs).squeeze()
    #
    #     distances = distances_i
    #     idxs = idxs_i
    #     if flip: # reverse temporal order
    #         patchlet_feats = torch.flip(patchlet_feats, [1])
    #         patchlet_points = torch.flip(patchlet_points, [1])
    #         idxs = torch.flip(idxs, [1])
    #         distances = torch.flip(distances, [1])
    #         patchlets = torch.flip(patchlets, [1])
    #
    #     patchlet_feats = patchlet_feats.reshape(b * t, n, self.k, d_feat)
    #     patchlet_points = patchlet_points.reshape(b * t, n, self.k, d)
    #     idxs = idxs.reshape(b * t, n, self.k)
    #     distances = distances.reshape(b * t, n, self.k)
    #     patchlets = patchlets.reshape(b * t, n, self.k)
    #
    #     fps_idx = None
    #     if self.downsample_method == 'fps':
    #         # select a subset of the points using fps for maximum coverage
    #         selected_idxs = utils.farthest_point_sample(x1[:, 0].contiguous(), self.npoints).to(torch.int64)
    #         selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, self.npoints)
    #         patchlet_points = utils.index_points(patchlet_points, selected_idxs)
    #         patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
    #         distances = utils.index_points(distances, selected_idxs)
    #         idxs = utils.index_points(idxs, selected_idxs)
    #         patchlets = utils.index_points(patchlets, selected_idxs)
    #         n = self.npoints
    #     elif self.downsample_method == 'var' or self.downsample_method == 'mean_var_t':
    #         # select a subset of the points with the largest point variance for maximum temporal movement
    #         if self.downsample_method == 'var':
    #             temporal_patchlet_points = patchlet_points.reshape(b, t, n, self.k, d).\
    #                 permute(0, 2, 1, 3, 4).reshape(b, n,-1, d)
    #             patchlet_variance = torch.linalg.norm(torch.var(temporal_patchlet_points, -2), dim=-1)
    #         elif self.downsample_method == 'mean_var_t':
    #             patchlet_variance = torch.linalg.norm(
    #                 torch.var(torch.mean(patchlet_points.reshape(b, t, n, self.k, d), -2), 1), dim=-1)
    #         else:
    #             raise ValueError("downsample method not supported ")
    #         _, selected_idxs = torch.topk(patchlet_variance, self.npoints)
    #         selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, self.npoints)
    #         patchlet_points = utils.index_points(patchlet_points, selected_idxs)
    #         patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
    #         distances = utils.index_points(distances, selected_idxs)
    #         idxs = utils.index_points(idxs, selected_idxs)
    #         patchlets = utils.index_points(patchlets, selected_idxs)
    #         n = self.npoints
    #
    #     return patchlet_points, patchlet_feats, distances, idxs, patchlets, n, d_feat, fps_idx, out_x

    def forward(self, point_seq, feat_seq=None):
        b, t, n, d = point_seq.shape
        n_original = n
        n_out = n

        # forward patchlets
        x1 = point_seq
        x2 = torch.cat([point_seq[:, [0]], point_seq], dim=1)[:, :-1]
        patchlet_points1, patchlet_feats1, distances1, idxs1, patchlets1, n, d_feat, fps_idx, out_x1 = \
            get_tpatches(x1, x2, feat_seq, flip=False,
                              k=self.k, radius=self.radius, res=self.res, sample_mode=self.sample_mode,
                              add_centroid_jitter=self.add_centroid_jitter,
                              downsample_method=self.downsample_method, npoints=self.npoints )

        #backward patchlets
        x1 = torch.flip(point_seq, [1])
        x2 = torch.cat([x1[:, [0]], x1], dim=1)[:, :-1]
        patchlet_points2, patchlet_feats2, distances2, idxs2, patchlets2, _, _, fps_idx2, out_x2 = \
            get_tpatches(x1, x2, feat_seq, flip=True,
                              k=self.k, radius=self.radius, res=self.res, sample_mode=self.sample_mode,
                              add_centroid_jitter=self.add_centroid_jitter,
                              downsample_method=self.downsample_method, npoints=self.npoints )

        # randomly select a subset
        rand_idxs = torch.randperm(n)[:int(n/2)]
        patchlets = torch.cat([patchlets1[:, rand_idxs, :], patchlets2[:, rand_idxs, :]], 1)
        patchlet_feats = torch.cat([patchlet_feats1[:, rand_idxs, :], patchlet_feats2[:, rand_idxs, :]], 1)
        patchlet_points = torch.cat([patchlet_points1[:, rand_idxs, :], patchlet_points2[:, rand_idxs, :]], 1)
        distances = torch.cat([distances1[:, rand_idxs, :], distances2[:, rand_idxs, :]], 1)
        idxs = torch.cat([idxs1[:, rand_idxs, :], idxs2[:, rand_idxs, :]], 1)
        out_x = out_x1 # remove after debug, out_x is unused

        # reshape all to bxtxnxk
        distances, idxs = distances.reshape(b, t, n, self.k), idxs.reshape(b, t, n, self.k)
        patchlets, patchlet_points = patchlets.reshape(b, t, n, self.k), patchlet_points.reshape(b, t, n, self.k, d)
        patchlet_feats = patchlet_feats.reshape(b, t, n, self.k, d_feat)

        normalized_patchlet_points = patchlet_points - patchlet_points[:, 0, :, [0], :].unsqueeze(1).detach() # normalize the patchlet around the center point of the first frame
        patchlet_feats = torch.cat([patchlet_feats, normalized_patchlet_points], -1)

        return {'idx': idxs, 'distances': distances, 'patchlets': patchlets,
                'patchlet_points': patchlet_points, 'patchlet_feats': patchlet_feats,
                'normalized_patchlet_points': normalized_patchlet_points, 'fps_idx': fps_idx,
                'x_current': out_x.reshape(b, t, n_out, d)}



class TPatchExtractorStrided(nn.Module):
    def __init__(self, k=16, sample_mode='nn', npoints=None, add_centroid_jitter=None, downsample_method=None,
                 radius=None, temporal_stride=8):
        super(TPatchExtractorStrided, self).__init__()
        self.k = k
        self.radius = radius
        self.sample_mode = sample_mode
        self.downsample_method = downsample_method
        self.npoints = npoints
        self.add_centroid_jitter = add_centroid_jitter
        self.res = faiss.StandardGpuResources()
        self.res.setDefaultNullStreamAllDevices()
        self.temporal_stride = temporal_stride

    # def get_tpatches(self, x1, x2, feat_seq, flip=False):
    #
    #     b, t, n, d = x1.shape
    #     n_out = n
    #
    #     if feat_seq is None:
    #         feat_seq = x1
    #         d_feat = d
    #     else:
    #         d_feat = feat_seq.shape[-1]
    #
    #     out_x = torch.empty(b, t, n_out, d)
    #     patchlets = torch.empty(b, t, n, self.k, device=x1.device, dtype=torch.long)
    #     distances_i = torch.empty(b, t, n, self.k, device=x1.device)
    #     idxs_i = torch.empty(b, t, n, self.k, device=x1.device, dtype=torch.long)
    #     patchlet_points = torch.empty(b, t, n, self.k, d, device=x1.device)
    #     patchlet_feats = torch.empty(b, t, n, self.k, d_feat, device=x1.device)
    #
    #     # loop over the data to reorder the indices to form the patchlets
    #     x_current = x2[:, 0]
    #     feat_seq_2 = torch.cat([feat_seq[:, [0]], feat_seq], dim=1)[:, :-1]
    #     for i in range(0, t):
    #         x_next = x1[:, i]
    #         distances, idxs = get_knn(x_current[..., 0:3], x_next[..., 0:3], k=self.k, res=self.res, method='keops',
    #                                   radius=self.radius)
    #         if self.sample_mode == 'nn':
    #             x_current = utils.index_points(x_next, idxs)[:, :, 0, :]
    #         elif self.sample_mode == 'randn':
    #             rand_idx = torch.randint(self.k, (b, n, 1), device=x_next.device, dtype=torch.int64).repeat(
    #                 [1, 1, 3]).unsqueeze(2)
    #             x_current = torch.gather(utils.index_points(x_next, idxs).squeeze(), dim=2, index=rand_idx).squeeze()
    #         elif self.sample_mode == 'gt':
    #             # only works when point correspondence is known and points are already aligned
    #             # if self.downsample_method == 'fps':
    #             #     x_current = utils.index_points(x_next, fps_idx).contiguous()
    #             # else:
    #             x_current = x_next
    #         else:
    #             raise ValueError("sample mode not supported")
    #
    #         out_x[:, i] = x_current
    #         if self.add_centroid_jitter is not None  and not self.sample_mode == 'gt':
    #             x_current = x_current + self.add_centroid_jitter * torch.randn_like(x_current)
    #
    #         distances_i[:, i], idxs_i[:, i] = distances, idxs
    #         patchlets[:, i] = idxs_i[:, i]
    #         patchlet_points[:, i] = utils.index_points(x_next, idxs).squeeze()
    #         patchlet_feats[:, i] = utils.index_points(feat_seq_2[:, i], idxs).squeeze()
    #
    #     distances = distances_i
    #     idxs = idxs_i
    #     if flip: # reverse temporal order
    #         patchlet_feats = torch.flip(patchlet_feats, [1])
    #         patchlet_points = torch.flip(patchlet_points, [1])
    #         idxs = torch.flip(idxs, [1])
    #         distances = torch.flip(distances, [1])
    #         patchlets = torch.flip(patchlets, [1])
    #
    #     patchlet_feats = patchlet_feats.reshape(b * t, n, self.k, d_feat)
    #     patchlet_points = patchlet_points.reshape(b * t, n, self.k, d)
    #     idxs = idxs.reshape(b * t, n, self.k)
    #     distances = distances.reshape(b * t, n, self.k)
    #     patchlets = patchlets.reshape(b * t, n, self.k)
    #
    #     fps_idx = None
    #     if self.downsample_method == 'fps':
    #         # select a subset of the points using fps for maximum coverage
    #         selected_idxs = utils.farthest_point_sample(x1[:, 0].contiguous(), self.npoints).to(torch.int64)
    #         selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, self.npoints)
    #         patchlet_points = utils.index_points(patchlet_points, selected_idxs)
    #         patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
    #         distances = utils.index_points(distances, selected_idxs)
    #         idxs = utils.index_points(idxs, selected_idxs)
    #         patchlets = utils.index_points(patchlets, selected_idxs)
    #         n = self.npoints
    #     elif self.downsample_method == 'var' or self.downsample_method == 'mean_var_t':
    #         # select a subset of the points with the largest point variance for maximum temporal movement
    #         if self.downsample_method == 'var':
    #             temporal_patchlet_points = patchlet_points.reshape(b, t, n, self.k, d).\
    #                 permute(0, 2, 1, 3, 4).reshape(b, n,-1, d)
    #             patchlet_variance = torch.linalg.norm(torch.var(temporal_patchlet_points, -2), dim=-1)
    #         elif self.downsample_method == 'mean_var_t':
    #             patchlet_variance = torch.linalg.norm(
    #                 torch.var(torch.mean(patchlet_points.reshape(b, t, n, self.k, d), -2), 1), dim=-1)
    #         else:
    #             raise ValueError("downsample method not supported ")
    #         _, selected_idxs = torch.topk(patchlet_variance, self.npoints)
    #         selected_idxs = selected_idxs.unsqueeze(1).repeat([1, t, 1]).reshape(-1, self.npoints)
    #         patchlet_points = utils.index_points(patchlet_points, selected_idxs)
    #         patchlet_feats = utils.index_points(patchlet_feats, selected_idxs)
    #         distances = utils.index_points(distances, selected_idxs)
    #         idxs = utils.index_points(idxs, selected_idxs)
    #         patchlets = utils.index_points(patchlets, selected_idxs)
    #         n = self.npoints
    #
    #     return patchlet_points, patchlet_feats, distances, idxs, patchlets, n, d_feat, fps_idx, out_x

    def forward(self, point_seq, feat_seq=None):
        b, t, n, d = point_seq.shape
        n_original = n
        n_out = n

        assert t % self.temporal_stride == 0
        n_temporal_segments = int(t / self.temporal_stride)
        patchlets_accum, patchlet_points_accum, patchlet_feats_accum, distances_accum,\
            idxs_accum, out_x_accum = [], [], [], [], [], []
        for i in range(n_temporal_segments):
            x1 = point_seq[:, i*self.temporal_stride:(i+1)*self.temporal_stride]
            x2 = torch.cat([x1[:, [0]], x1], dim=1)[:, :-1]
            feats = feat_seq[:, i*self.temporal_stride:(i+1)*self.temporal_stride] if feat_seq is not None else None
            patchlet_points, patchlet_feats, distances, idxs, patchlets, n, d_feat, fps_idx, out_x = \
                get_tpatches(x1, x2, feats, flip=False,
                              k=self.k, radius=self.radius, res=self.res, sample_mode=self.sample_mode,
                              add_centroid_jitter=self.add_centroid_jitter,
                              downsample_method=self.downsample_method, npoints=self.npoints )

            #bidirectional
            x1 = torch.flip(x1, [1])
            x2 = torch.cat([x1[:, [0]], x1], dim=1)[:, :-1]
            patchlet_points2, patchlet_feats2, distances2, idxs2, patchlets2, _, _, fps_idx2, out_x2 = \
                get_tpatches(x1, x2, feat_seq, flip=True,
                              k=self.k, radius=self.radius, res=self.res, sample_mode=self.sample_mode,
                              add_centroid_jitter=self.add_centroid_jitter,
                              downsample_method=self.downsample_method, npoints=self.npoints )

            # randomly select a subset
            rand_idxs = torch.randperm(n)[:int(n / 2)]
            patchlets = torch.cat([patchlets[:, rand_idxs, :], patchlets2[:, rand_idxs, :]], 1)
            patchlet_feats = torch.cat([patchlet_feats[:, rand_idxs, :], patchlet_feats2[:, rand_idxs, :]], 1)
            patchlet_points = torch.cat([patchlet_points[:, rand_idxs, :], patchlet_points2[:, rand_idxs, :]], 1)
            distances = torch.cat([distances[:, rand_idxs, :], distances2[:, rand_idxs, :]], 1)
            idxs = torch.cat([idxs[:, rand_idxs, :], idxs2[:, rand_idxs, :]], 1)
            out_x = out_x  # remove after debug, out_x is unused

            patchlets_accum.append(patchlets.reshape(b, self.temporal_stride, n, self.k))
            patchlet_points_accum.append(patchlet_points.reshape(b, self.temporal_stride, n, self.k, d))
            patchlet_feats_accum.append(patchlet_feats.reshape(b, self.temporal_stride, n, self.k, d_feat))
            distances_accum.append(distances.reshape(b, self.temporal_stride, n, self.k))
            idxs_accum.append(idxs.reshape(b, self.temporal_stride, n, self.k))
            out_x_accum.append(out_x.reshape(b, self.temporal_stride, n_original, d))

        patchlets = torch.cat(patchlets_accum, 1)
        patchlet_points = torch.cat(patchlet_points_accum, 1)
        patchlet_feats = torch.cat(patchlet_feats_accum, 1)
        distances = torch.cat(distances_accum, 1)
        idxs = torch.cat(idxs_accum, 1)
        out_x = torch.cat(out_x_accum, 1)

        normalized_patchlet_points = patchlet_points.detach().clone()
        for i in range(n_temporal_segments):
            normalized_patchlet_points[:, i*self.temporal_stride:(i+1)*self.temporal_stride] -= \
                normalized_patchlet_points[:, i * self.temporal_stride, :, [0], :].unsqueeze(1).detach()
        patchlet_feats = torch.cat([patchlet_feats, normalized_patchlet_points], -1)

        return {'idx': idxs, 'distances': distances, 'patchlets': patchlets,
                'patchlet_points': patchlet_points, 'patchlet_feats': patchlet_feats,
                'normalized_patchlet_points': normalized_patchlet_points, 'fps_idx': fps_idx,
                'x_current': out_x.reshape(b, t, n_out, d)}



class TPatchExtractorModStrided(nn.Module):
    def __init__(self, k=16, sample_mode='nn', npoints=None, add_centroid_jitter=None, downsample_method=None,
                 radius=None, temporal_stride=8):
        super(TPatchExtractorModStrided, self).__init__()
        self.k = k
        self.radius = radius
        self.sample_mode = sample_mode
        self.downsample_method = downsample_method
        self.npoints = npoints
        self.add_centroid_jitter = add_centroid_jitter
        self.res = faiss.StandardGpuResources()
        self.res.setDefaultNullStreamAllDevices()
        self.temporal_stride = temporal_stride
    def forward(self, point_seq, feat_seq=None):
        b, t, n, d = point_seq.shape
        n_original = n
        n_out = n

        distances, idxs = get_knn(point_seq[..., n, 0:3], point_seq[..., n, 0:3], k=self.k, res=self.res, method='keops',
                                  radius=self.radius)
        point_patches = utils.index_points(point_seq, idxs)[:, :, 0, :]

        # assert t % self.temporal_stride == 0
        # n_temporal_segments = int(t / self.temporal_stride)
        # patchlets_accum, patchlet_points_accum, patchlet_feats_accum, distances_accum,\
        #     idxs_accum, out_x_accum = [], [], [], [], [], []
        # for i in range(n_temporal_segments):
        #     x1 = point_seq[:, i*self.temporal_stride:(i+1)*self.temporal_stride]
        #     x2 = torch.cat([x1[:, [0]], x1], dim=1)[:, :-1]
        #     feats = feat_seq[:, i*self.temporal_stride:(i+1)*self.temporal_stride] if feat_seq is not None else None
        #     patchlet_points, patchlet_feats, distances, idxs, patchlets, n, d_feat, fps_idx, out_x = \
        #         get_tpatches(x1, x2, feats, flip=False,
        #                       k=self.k, radius=self.radius, res=self.res, sample_mode=self.sample_mode,
        #                       add_centroid_jitter=self.add_centroid_jitter,
        #                       downsample_method=self.downsample_method, npoints=self.npoints )
        #
        #     #bidirectional
        #     x1 = torch.flip(x1, [1])
        #     x2 = torch.cat([x1[:, [0]], x1], dim=1)[:, :-1]
        #     patchlet_points2, patchlet_feats2, distances2, idxs2, patchlets2, _, _, fps_idx2, out_x2 = \
        #         get_tpatches(x1, x2, feat_seq, flip=True,
        #                       k=self.k, radius=self.radius, res=self.res, sample_mode=self.sample_mode,
        #                       add_centroid_jitter=self.add_centroid_jitter,
        #                       downsample_method=self.downsample_method, npoints=self.npoints )
        #
        #     # randomly select a subset
        #     rand_idxs = torch.randperm(n)[:int(n / 2)]
        #     patchlets = torch.cat([patchlets[:, rand_idxs, :], patchlets2[:, rand_idxs, :]], 1)
        #     patchlet_feats = torch.cat([patchlet_feats[:, rand_idxs, :], patchlet_feats2[:, rand_idxs, :]], 1)
        #     patchlet_points = torch.cat([patchlet_points[:, rand_idxs, :], patchlet_points2[:, rand_idxs, :]], 1)
        #     distances = torch.cat([distances[:, rand_idxs, :], distances2[:, rand_idxs, :]], 1)
        #     idxs = torch.cat([idxs[:, rand_idxs, :], idxs2[:, rand_idxs, :]], 1)
        #     out_x = out_x  # remove after debug, out_x is unused
        #
        #     patchlets_accum.append(patchlets.reshape(b, self.temporal_stride, n, self.k))
        #     patchlet_points_accum.append(patchlet_points.reshape(b, self.temporal_stride, n, self.k, d))
        #     patchlet_feats_accum.append(patchlet_feats.reshape(b, self.temporal_stride, n, self.k, d_feat))
        #     distances_accum.append(distances.reshape(b, self.temporal_stride, n, self.k))
        #     idxs_accum.append(idxs.reshape(b, self.temporal_stride, n, self.k))
        #     out_x_accum.append(out_x.reshape(b, self.temporal_stride, n_original, d))
        #
        # patchlets = torch.cat(patchlets_accum, 1)
        # patchlet_points = torch.cat(patchlet_points_accum, 1)
        # patchlet_feats = torch.cat(patchlet_feats_accum, 1)
        # distances = torch.cat(distances_accum, 1)
        # idxs = torch.cat(idxs_accum, 1)
        # out_x = torch.cat(out_x_accum, 1)
        #
        # normalized_patchlet_points = patchlet_points.detach().clone()
        # for i in range(n_temporal_segments):
        #     normalized_patchlet_points[:, i*self.temporal_stride:(i+1)*self.temporal_stride] -= \
        #         normalized_patchlet_points[:, i * self.temporal_stride, :, [0], :].unsqueeze(1).detach()
        # patchlet_feats = torch.cat([patchlet_feats, normalized_patchlet_points], -1)

        return {'idx': idxs, 'distances': distances, 'patchlets': patchlets,
                'patchlet_points': patchlet_points, 'patchlet_feats': patchlet_feats,
                'normalized_patchlet_points': normalized_patchlet_points, 'fps_idx': fps_idx,
                'x_current': out_x.reshape(b, t, n_out, d)}
                """

# Modify extractors.py.
with open('../models/extractors.py', 'w') as python_file:
  python_file.writelines(extractors_py)

# run_MSR-Action3D_experiment.sh

In [ ]:
run_MSR_Action3D_experiment_script = """
#!/usr/bin/env bash

GPU_IDX=0
export CUDA_DEVICE_ORDER="PCI_BUS_ID"
export CUDA_VISIBLE_DEVICES=$GPU_IDX

IDENTIFIER='debug'
CONFIG='configs/msr-action3d/config_msr_action3d.yaml'
LOGDIR='./log/'

python train.py --identifier $IDENTIFIER --config $CONFIG --logdir $LOGDIR --fix_random_seed
# python test.py --identifier $IDENTIFIER --model_ckpt '000200.pt' --logdir $LOGDIR --fix_random_seed
# python ./evaluate.py --identifier $IDENTIFIER --logdir $LOGDIR
"""

with open('../run_MSR-Action3D_experiment.sh', 'w') as python_file:
  python_file.writelines(run_MSR_Action3D_experiment_script)

# Move up to the parent directory.
change_directory('..')

# Install Python Packages

In [ ]:
!pip install ninja faiss-gpu faiss-cpu pykeops wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for pykeops: filename=pykeops-2.2.3-py3-none-any.whl size=118640 sha256=2f66935d22791ce5

# train.py

In [ ]:
change_directory('/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction')

In [ ]:
train_py = """
# Author: Yizhak Ben-Shabat (Itzik), 2022
# train 3DInAction

import os
import yaml
import argparse
import i3d_utils as utils
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random

from models.pointnet import feature_transform_regularizer
from models import build_model
from datasets import build_dataloader

import wandb
from tqdm import tqdm


parser = argparse.ArgumentParser()
parser.add_argument('--logdir', type=str, default='./log/', help='path to model save dir')
parser.add_argument('--identifier', type=str, default='debug', help='unique run identifier')
parser.add_argument('--config', type=str, default='./configs/dfaust/config_dfaust.yaml', help='path to yaml config file')
parser.add_argument('--fix_random_seed', action='store_true', default=False, help='fix random seed')
args = parser.parse_args()


def main():
    cfg = yaml.safe_load(open(args.config))
    logdir = os.path.join(args.logdir, args.identifier)
    os.makedirs(logdir, exist_ok=True)

    # TODO: move to cfg project_name, entity
    if cfg['DATA'].get('name') == 'DFAUST':
        project_name = 'DFAUST'
    elif cfg['DATA'].get('name') == 'IKEA_EGO':
        project_name = 'IKEA EGO'
    elif cfg['DATA'].get('name') == 'IKEA_ASM':
        project_name = 'IKEA ASM'
    elif cfg['DATA'].get('name') == 'MSR-Action3D':
        project_name = 'MSR-Action3D'
    else:
        raise NotImplementedError
    wandb_run = wandb.init(project=project_name, entity='mkjohn', save_code=True)
    cfg['WANDB'] = {'id': wandb_run.id, 'project': wandb_run.project, 'entity': wandb_run.entity}

    with open(os.path.join(logdir, 'config.yaml'), 'w') as outfile:
        yaml.dump(cfg, outfile, default_flow_style=False)

    wandb_run.name = args.identifier
    wandb.config.update(cfg)  # adds all the arguments as config variables
    wandb.run.log_code(".")
    # define our custom x axis metric
    wandb.define_metric("train/step")
    wandb.define_metric("train/*", step_metric="train/step")
    wandb.define_metric("test/*", step_metric="train/step")

    # need to add argparse
    run(cfg, logdir)

def run(cfg, logdir):
    n_epochs = cfg['TRAINING']['n_epochs']
    lr = cfg['TRAINING']['lr']
    batch_size = cfg['TRAINING']['batch_size']
    refine, refine_epoch = cfg['TRAINING']['refine'], cfg['TRAINING']['refine_epoch']
    pretrained_model = cfg['TRAINING']['pretrained_model']
    pc_model = cfg['MODEL']['pc_model']
    frames_per_clip = cfg['DATA']['frames_per_clip']
    num_steps_per_update = cfg['TRAINING']['steps_per_update']
    save_every = cfg['save_every']

    if args.fix_random_seed:
        seed = cfg['seed']
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    os.system('cp "%s" "%s"' % (__file__, logdir))  # backup the current training file
    os.makedirs(os.path.join(logdir, 'models'), exist_ok=True)
    #os.system("cp '%s' '%s'" % ('/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/models/*.py', os.path.join(logdir, 'models')))  # backup the models files

    # build dataloader and dataset
    train_dataloader, train_dataset = build_dataloader(config=cfg, training=True, shuffle=False) # should be unshuffled because of sampler
    test_dataloader, test_dataset = build_dataloader(config=cfg, training=False, shuffle=True)

    num_classes = train_dataset.num_classes

    # build model
    model = build_model(cfg['MODEL'], num_classes, frames_per_clip)

    if pretrained_model is not None:
        checkpoints = torch.load(pretrained_model)
        model.load_state_dict(checkpoints["model_state_dict"])  # load trained model
        model.replace_logits(num_classes)

    if refine:
        if refine_epoch == 0:
            raise ValueError("You set the refine epoch to 0. No need to refine, just retrain.")
        refine_model_filename = os.path.join(logdir, str(refine_epoch).zfill(6)+'.pt')
        checkpoint = torch.load(refine_model_filename)
        model.load_state_dict(checkpoint["model_state_dict"])

    model.cuda()
    model = nn.DataParallel(model)

    best_model_trained_yet_and_its_accuracy = [None, 51.25]

    # define optimizer and scheduler
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1E-6)
    lr_sched = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.5)

    if refine:
        lr_sched.load_state_dict(checkpoint["lr_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    steps = 0
    n_examples = 0
    train_num_batch = len(train_dataloader)
    test_num_batch = len(test_dataloader)
    refine_flag = True

    pbar = tqdm(total=n_epochs, desc='Training', dynamic_ncols=True)
    pbar.update(refine_epoch)
    while steps <= n_epochs:
        if steps <= refine_epoch and refine and refine_flag:
            # lr_sched.step()
            steps += 1
            n_examples += len(train_dataset.clip_set)
            continue
        else:
            refine_flag = False
        # Each epoch has a training and validation phase

        test_batchind = -1
        test_fraction_done = 0.0
        test_enum = enumerate(test_dataloader, 0)
        tot_loss = 0.0
        tot_loc_loss = 0.0
        tot_cls_loss = 0.0
        num_iter = 0
        optimizer.zero_grad()

        # Iterate over data.
        avg_acc = []
        loader_pbar = tqdm(total=len(train_dataloader), dynamic_ncols=True, leave=False)
        for train_batchind, data in enumerate(train_dataloader):
            num_iter += 1
            # get the inputs
            inputs = data[1]
            labels = data[0]
            # inputs, labels, vid_idx, frame_pad = data['inputs'], data['labels'], data['vid_idx'], data['frame_pad']
            in_channel = cfg['MODEL'].get('in_channel', 3)
            inputs = inputs[:, :, 0:in_channel, :]
            inputs = inputs.cuda().requires_grad_().contiguous()
            labels = labels.cuda()

            out_dict = model(inputs)
            per_frame_logits = out_dict['pred']

            labels = labels.unsqueeze(1) + torch.zeros((per_frame_logits.shape[0], per_frame_logits.shape[2])).cuda()
            labels = labels.to(dtype = torch.long)

            # compute localization loss
            loc_loss = F.cross_entropy(per_frame_logits, labels.to(dtype = torch.long))
            tot_loc_loss += loc_loss.item()

            # compute classification loss (with max-pooling along time dim1 x dim2)
            cls_loss = F.cross_entropy(torch.max(per_frame_logits, dim=2)[0], torch.max(labels, dim=1)[0])
            tot_cls_loss += cls_loss.item()
            loss = (0.5 * loc_loss + 0.5 * cls_loss) / num_steps_per_update
            if pc_model == 'pn1' or pc_model == 'pn1_4d_basic':
                trans, trans_feat = out_dict['trans'], out_dict['trans_feat']
                loss += 0.001 * feature_transform_regularizer(trans) + 0.001 * feature_transform_regularizer(trans_feat)

            tot_loss += loss.item()
            loss.backward()

            acc = utils.accuracy_v2(torch.argmax(per_frame_logits, dim=1), labels)
            avg_acc.append(acc.item())

            train_fraction_done = (train_batchind + 1) / train_num_batch

            if num_iter == num_steps_per_update or train_batchind == len(train_dataloader)-1:
                n_steps = num_steps_per_update
                if train_batchind == len(train_dataloader)-1:
                    n_steps = num_iter
                n_examples += batch_size*n_steps
                optimizer.step()
                optimizer.zero_grad()
                # log train losses
                log_dict = {
                    "train/step": n_examples,
                    "train/loss": tot_loss / n_steps,
                    "train/cls_loss": tot_cls_loss / n_steps,
                    "train/loc_loss": tot_loc_loss / n_steps,
                    "train/Accuracy": np.mean(avg_acc),
                    "train/lr":  optimizer.param_groups[0]['lr'],
                    "train/epoch": steps,
                }
                wandb.log(log_dict)

                num_iter = 0
                tot_loss = 0.

            if test_fraction_done <= train_fraction_done and test_batchind + 1 < test_num_batch:
                model.eval()
                test_batchind, data = next(test_enum)
                inputs = data[1]
                labels = data[0]
                # inputs, labels, vid_idx, frame_pad = data['inputs'], data['labels'], data['vid_idx'], data['frame_pad']
                in_channel = cfg['MODEL'].get('in_channel', 3)
                inputs = inputs[:, :, 0:in_channel, :]
                inputs = inputs.cuda().requires_grad_().contiguous()
                labels = labels.cuda()

                with torch.no_grad():
                    out_dict = model(inputs)
                    per_frame_logits = out_dict['pred']

                    labels = labels.unsqueeze(1) + torch.zeros((per_frame_logits.shape[0], per_frame_logits.shape[2])).cuda()
                    labels = labels.to(dtype = torch.long)

                    # compute localization loss
                    loc_loss = F.cross_entropy(per_frame_logits, labels)
                    # compute classification loss (with max-pooling along time dim1 x dim2)
                    cls_loss = F.cross_entropy(torch.max(per_frame_logits, dim=2)[0],
                                                                  torch.max(labels, dim = 1)[0])
                    loss = (0.5 * loc_loss + 0.5 * cls_loss) / num_steps_per_update
                    if pc_model == 'pn1' or pc_model == 'pn1_4d_basic':
                        trans, trans_feat = out_dict['trans'], out_dict['trans_feat']
                        loss += (0.001 * feature_transform_regularizer(trans) +
                                 0.001 * feature_transform_regularizer(trans_feat)) / num_steps_per_update
                    acc = utils.accuracy_v2(torch.argmax(per_frame_logits, dim=1), labels)

                log_dict = {
                    "test/step": n_examples,
                    "test/loss": loss.item(),
                    "test/cls_loss": loc_loss.item(),
                    "test/loc_loss": cls_loss.item(),
                    "test/Accuracy": acc.item()
                }
                wandb.log(log_dict)
                test_fraction_done = (test_batchind + 1) / test_num_batch
                model.train()

            loader_pbar.update()
        loader_pbar.close()

        if steps % save_every == 0 or steps == n_epochs:
            # save model
            torch.save({"model_state_dict": model.module.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "lr_state_dict": lr_sched.state_dict()},
                       os.path.join(logdir, str(steps).zfill(6) + '.pt'))

        steps += 1
        lr_sched.step()
        pbar.update()


if __name__ == '__main__':
    main()

"""
with open('train.py', 'w') as python_file:
  python_file.writelines(train_py)

In [ ]:
# !python models/setup.py install

# Run the Experiment Using the MSR-Action3D Dataset

In [ ]:
#import os
#os.chdir('/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/')

In [ ]:
#!./run_experiment.sh

In [ ]:
!chmod u=rwx run_MSR-Action3D_experiment.sh

In [ ]:
!git branch -l
!git branch

* dev
  dev-mathew
* dev
  dev-mathew


In [ ]:
!./run_MSR-Action3D_experiment.sh

DEBUG
DEBUG
DEBUG
DEBUG
DEBUG-22
DEBUG-27
DEBUG-31
DEBUG
wandb: Currently logged in as: mkjohn. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/wandb/run-20240504_200630-3d11vv6l
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run old-lightsaber-13
wandb: ⭐️ View project at https://wandb.ai/mkjohn/MSR-Action3D
wandb: 🚀 View run at https://wandb.ai/mkjohn/MSR-Action3D/runs/3d11vv6l
Number of clips in the train set:1555
Number of clips in the test set:1038
Training:   0% 0/3000 [00:00<?, ?it/s]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:00<00:00,  2.84it/s]
Training:  90% 2701/3000 [00:00<00:00, 7599.97it/s]
  0% 0/1 [00:00<?, ?it/s]
                         
  0% 0/1 [00:00<?, ?it/s]
                         
  0% 0/1 [00:00<?, ?it/s]
                         
  0% 0/1 [00:00<?, ?it/s]
                

# Save wandb files to Google Drive.

In [ ]:
yaml_document['TRAINING']['n_epochs'] -= 300
yaml_document['TRAINING']['refine_epoch'] -= 300

In [ ]:
file_1 = "/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/MSRAction3DFPS/log/debug/{:06d}.pt".format(yaml_document['TRAINING']['n_epochs'])
file_2 = "/content/drive/My Drive/Colab Notebooks/COMPSCI 674/Final Project/CS-674-Final-Project-3dInAction/log/debug/{:06d}.pt".format(yaml_document['TRAINING']['n_epochs'])

copy_file(file_2, file_1)

yaml_document['TRAINING']['n_epochs'] += 300
yaml_document['TRAINING']['refine_epoch'] += 300

out_file_path = '/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/MSRAction3DFPS/configs/msr-action3d/'

newdir(out_file_path, exist_ok = True)

with open(out_file_path + 'config_msr_action3d.yaml', 'w') as yaml_file:
  yaml.safe_dump(yaml_document, yaml_file)

In [ ]:
!git branch -c dev dev-mathew

In [ ]:
!git branch --list

* dev
  dev-mathew


In [ ]:
!git config --system --add user.name GitUser5877857139983029

In [ ]:
!git config --list --system

filter.lfs.required=true
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process


In [ ]:
!git pull origin

Already up to date.


In [ ]:
!cat .git/config

[core]
	repositoryformatversion = 0
	filemode = true
	bare = false
	logallrefupdates = true
[remote "origin"]
	url = https://github.com/EfranH25/CS-674-Final-Project-3dInAction.git
	fetch = +refs/heads/*:refs/remotes/origin/*
[branch "dev"]
	remote = origin
	merge = refs/heads/dev
[branch "dev-mathew"]
	remote = origin
	merge = refs/heads/dev
[gui]
	wmstate = zoomed
	geometry = 1109x563+224+224 216 255
[user]
	name = GitUser5877857139983029


In [ ]:
!(type -p wget >/dev/null || (sudo apt update && sudo apt-get install wget -y)) \
&& sudo mkdir -p -m 755 /etc/apt/keyrings \
&& wget -qO- https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo tee /etc/apt/keyrings/githubcli-archive-keyring.gpg > /dev/null \
&& sudo chmod go+r /etc/apt/keyrings/githubcli-archive-keyring.gpg \
&& echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null \
&& sudo apt update \
&& sudo apt install gh -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:12 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,371 kB

In [ ]:
!sudo apt update
!sudo apt install gh

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,371 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,756 kB]
Hit:13 https://ppa.launchpa

In [ ]:
!gh status

To get started with GitHub CLI, please run:  gh auth login
Alternatively, populate the GH_TOKEN environment variable with a GitHub API authentication token.


In [ ]:
!echo "my secret API key" > my_token_file

In [ ]:
!gh auth login --with-token < my_token_file

In [ ]:
!gh

unknown command "status" for "gh"

Usage:  gh <command> <subcommand> [flags]

Available commands:
  actions
  alias
  api
  auth
  browse
  codespace
  completion
  config
  extension
  gist
  gpg-key
  help
  issue
  pr
  release
  repo
  run
  secret
  ssh-key
  workflow



In [ ]:
!gh repo list EfranH25


Showing 19 of 19 repositories in @EfranH25

NAME                      DESCRIPTION           INFO          UPDATED           
EfranH25/CS-674-Final...  This is the repos...  public, fork  about 1 day ago
EfranH25/Generative-A...                        public        about 4 months ago
EfranH25/Airbnb-Machi...                        public        about 4 months ago
EfranH25/IBM-Data-Sci...  This repo holds m...  public        about 1 year ago
EfranH25/ansible          Ansible is a radi...  public, fork  about 2 years ago
EfranH25/markdown-here    Google Chrome, Fi...  public, fork  about 2 years ago
EfranH25/Coursera-Pyt...                        public        about 2 years ago
EfranH25/Spoon-Knife      This repo is for ...  public, fork  about 2 years ago
EfranH25/API              Learn how to use ...  public, fork  about 3 years ago
EfranH25/Cleaning-Dat...  Cleaning Data for...  public, fork  about 3 years ago
EfranH25/Excel-Samples    This Repo is to d...  public        about 3 years

In [ ]:
!gh repo

Work with GitHub repositories.

USAGE
  gh repo <command> [flags]

GENERAL COMMANDS
  create:      Create a new repository
  list:        List repositories owned by user or organization

TARGETED COMMANDS
  archive:     Archive a repository
  clone:       Clone a repository locally
  delete:      Delete a repository
  deploy-key:  Manage deploy keys in a repository
  edit:        Edit repository settings
  fork:        Create a fork of a repository
  rename:      Rename a repository
  set-default: Configure default repository for this directory
  sync:        Sync a repository
  unarchive:   Unarchive a repository
  view:        View a repository

INHERITED FLAGS
  --help   Show help for command

ARGUMENTS
  A repository can be supplied as an argument in any of the following formats:
  - "OWNER/REPO"
  - by URL, e.g. "https://github.com/OWNER/REPO"

EXAMPLES
  $ gh repo create
  $ gh repo clone cli/cli
  $ gh repo view --web

LEARN MORE
  Use `gh <command> <subcommand> --help` for more

In [ ]:
!gh repo sync EfranH25/CS-674-Final-Project-3dInAction -b dev-mathew

✓ Synced the "EfranH25:main" branch from "sitzikbs:main"


In [ ]:
!git push https://github.com/EfranH25/CS-674-Final-Project-3dInAction.git dev-mathew

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git -c user.name=GitUser5877857139983029 push https://github.com/EfranH25/CS-674-Final-Project-3dInAction.git dev-mathew

fatal: could not read Username for 'https://github.com': No such device or address
